In [3]:
import pickle
import pandas as pd
import numpy as np
import os
from collections import defaultdict
import matplotlib.pyplot as plt


In [11]:
with open("WESAD/S3/S3_n0.pkl", 'rb') as file:
    data = pickle.load(file, encoding='bytes')
for device in ['wrist', 'chest']:
    for sensor in data[device]:
        print(f'{device}_{sensor}: {data[device][sensor].shape}')


wrist_ACC: (2930, 3, 32)
wrist_BVP: (2930, 1, 64)
wrist_EDA: (2930, 1, 4)
wrist_TEMP: (2930, 1, 4)
chest_ACC: (2930, 3, 700)
chest_ECG: (2930, 1, 700)
chest_EMG: (2930, 1, 700)
chest_EDA: (2930, 1, 700)
chest_Temp: (2930, 1, 700)
chest_Resp: (2930, 1, 700)


In [1]:
# Creating dataframe of all of them
# `window` | `mean` | `std` | `max` | `min` | `label`

from feature import extract_features

extract_features(verbose=1)


Data of S2 loaded!
Data of S3 loaded!
Data of S4 loaded!
Data of S5 loaded!
Data of S6 loaded!
Data of S7 loaded!
Data of S8 loaded!
Data of S9 loaded!
Data of S10 loaded!
Data of S11 loaded!
Data of S13 loaded!
Data of S14 loaded!
Data of S15 loaded!
Data of S16 loaded!
Data of S17 loaded!


In [5]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from training import WESADTrainer
import pandas as pd

subjects = {i for i in range(2, 18)}.difference({12})
trainer = WESADTrainer({
    RandomForestClassifier: 
        {'n_estimators': [100], 'min_samples_leaf': [1]},
    MLPClassifier:
        {},
    
    }, sensor='chest_Resp', subjects=subjects, data_prefix="svd", classes=[2])

reports = trainer.predict_all(verbose=3)


Testing S2
test #S2 skipped

input of size (42069, 6)
Training RandomForestClassifier() completed in 48.474334955215454 secs (score=0.7925797662887943)
Training MLPClassifier() completed in 43.78752422332764 secs (score=0.5353593643641084)
predicting df of size (2889, 5), made (2889,)
Report of S2: {'2': {'precision': 0.8801955990220048, 'recall': 0.5853658536585366, 'f1-score': 0.703125, 'support': 615}, '3': {'precision': 0.8971774193548387, 'recall': 0.9784520668425681, 'f1-score': 0.9360538493899875, 'support': 2274}, 'accuracy': 0.894773277950848, 'macro avg': {'precision': 0.8886865091884217, 'recall': 0.7819089602505523, 'f1-score': 0.8195894246949937, 'support': 2889}, 'weighted avg': {'precision': 0.8935623901043394, 'recall': 0.894773277950848, 'f1-score': 0.886468787993365, 'support': 2889}}
Testing S3
test #S3 skipped

input of size (42023, 6)
Training RandomForestClassifier() completed in 54.44404602050781 secs (score=0.7964324970362877)
Training MLPClassifier() completed 

In [6]:
for item in reports:
    print(f"{item}: {reports[item]}")

1: {'precision': 0, 'recall': 0, 'f1-score': 0}
2: {'precision': 0.6963993299941651, 'recall': 0.7224182369702018, 'f1-score': 0.6893291553323447}
3: {'precision': 0.9210738409342901, 'recall': 0.8945740858623811, 'f1-score': 0.9062676559483119}
4: {'precision': 0, 'recall': 0, 'f1-score': 0}
weighted avg: {'precision': 0.8711524471000265, 'recall': 0.8568923181308491, 'f1-score': 0.8584260810631972}
macro avg: {'precision': 0.8087365854642274, 'recall': 0.8084961614162914, 'f1-score': 0.7977984056403286}
accuracy: 0.8568923181308491


In [10]:
import pickle

with open("WESAD/S3/S3_n0.pkl", "rb") as file:
    s = pickle.load(file, encoding="bytes")

print(s.keys())

dict_keys(['wrist', 'chest', 'label'])


In [1]:
import pickle
import numpy as np

subject = 17
with open(f'WESAD/S{subject}/S{subject}_n0.pkl', 'rb') as pklfile:
    data = pickle.load(pklfile, encoding='bytes')

print(data.keys())
for device in ['wrist', 'chest']:
    for sensor in data[device].keys():
        print(f"{device}_{sensor}: {len(data[device][sensor])}")
print(f"labels: {data['label'].shape}")


arrays = [data[device][sensor].mean((1, 2)).reshape(1, -1) for device in ['wrist', 'chest'] for sensor in data[device].keys()]
print([a.shape for a in arrays])
concated = np.concatenate(arrays)
print(concated.shape)



dict_keys(['wrist', 'chest', 'label'])
wrist_ACC: 3002
wrist_BVP: 3002
wrist_EDA: 3002
wrist_TEMP: 3002
chest_ACC: 3002
chest_ECG: 3002
chest_EMG: 3002
chest_EDA: 3002
chest_Temp: 3002
chest_Resp: 3002
labels: (3002, 700)
[(1, 3002), (1, 3002), (1, 3002), (1, 3002), (1, 3002), (1, 3002), (1, 3002), (1, 3002), (1, 3002), (1, 3002)]
(10, 3002)


In [1]:
from biot.BIOT.run_binary_supervised import prepare_WESAD_dataloader_manual, supervised

class Args:
    def __init__(self):
        self.epochs = 100
        self.lr = 1e-3
        self.weight_decay = 1e-5
        self.batch_size = 4
        self.num_workers = 4
        self.dataset = "WESAD"
        self.model = "BIOT"
        self.in_channels = 16
        self.sample_length = 10
        self.n_classes = 1
        self.sampling_rate = 200
        self.token_size = 200
        self.hop_length = 100
        self.pretrain_model_path = ""
        self.server = "pc" # or colab
        pass

args = Args()
supervised(args)


Welcome
Dataset: WESAD
Nom of:
	Train: 10
	Val:   4
	Test:  1
window size = 10
window size = 10
window size = 10
Size of loader of:
	Train: 25
	Val:   10
	Test:  3


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type           | Params
-----------------------------------------
0 | model | BIOTClassifier | 3.2 M 
-----------------------------------------
3.2 M     Trainable params
16        Non-trainable params
3.2 M     Total params
12.746    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\Elham moin\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "c:\Users\Elham moin\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\utils\data\_utils\fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "c:\Users\Elham moin\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\utils\data\_utils\collate.py", line 316, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "c:\Users\Elham moin\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\utils\data\_utils\collate.py", line 173, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  File "c:\Users\Elham moin\Ap

In [2]:
for item, label in train:
    print(item.shape) # batch_size, channels, freqs
    print(label.shape) # batch_size, freqs
    break


torch.Size([2, 10, 3000])
torch.Size([2, 3000])
